In [3]:
# coding: UTF-8
import time
import torch
import numpy as np
from train_eval import train, init_network
from importlib import import_module
import pickle as pkl

dataset = 'dataset'  # 数据集
embedding = 'random'
model_name = 'TextCNN'  # 'TextRCNN'  # TextCNN, TextRNN, FastText, TextRCNN, TextRNN_Att, DPCNN, Transformer
x = import_module(model_name)
config = x.Config(dataset, embedding)

vocab = pkl.load(open(config.vocab_path, 'rb'))
config.n_vocab = len(vocab)
model = x.Model(config).to(config.device)

In [69]:
id_to_cate = {int(x.split(" +++$+++ ")[1].strip()):x.split(" +++$+++ ")[0] for x in open(dataset + '/data/id_to_cate.txt', encoding='utf-8').readlines()}

In [76]:
id_to_cate[2]

'口腔护理'

In [5]:
model.load_state_dict(torch.load(config.save_path))
model.eval()

Model(
  (embedding): Embedding(3454, 300, padding_idx=3453)
  (convs): ModuleList(
    (0): Conv2d(1, 256, kernel_size=(2, 300), stride=(1, 1))
    (1): Conv2d(1, 256, kernel_size=(3, 300), stride=(1, 1))
    (2): Conv2d(1, 256, kernel_size=(4, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=768, out_features=26, bias=True)
)

In [ ]:
with torch.no_grad():
    for texts, labels in data_iter:
        outputs = model(texts)

In [41]:
def load_dataset(content, pad_size=32):
    UNK, PAD = '<UNK>', '<PAD>'  # 未知字，padding符号
    tokenizer = lambda x: [y for y in x]
    contents = []
    words_line = []
    token = tokenizer(content)
    seq_len = len(token)
    if pad_size:
        if len(token) < pad_size:
            token.extend([PAD] * (pad_size - len(token)))
        else:
            token = token[:pad_size]
            seq_len = pad_size
    # word to id
    for word in token:
        words_line.append(vocab.get(word, vocab.get(UNK)))
    contents.append(words_line)
    return torch.LongTensor(contents),"placeholder"

In [48]:
content = load_dataset("禧贝happybaby 蓝莓甜菜糙米饼40g/袋")
outputs = model(content)
predic = torch.max(outputs.data, 1)[1].cpu().numpy()
print("outputs:",outputs)
print("predic:",predic)

outputs: tensor([[ -0.3162,   0.7501,  -6.2322,  -7.3522,  -6.8863,  -6.0743,  -4.0020,
          -3.6948,  -7.3698,   8.7794,  -3.1653,  -4.1497,  -1.3720,   1.6864,
           1.5171,  -7.0695,  -8.6547, -12.9463,  -8.0630,  -6.2699,  -3.6001,
         -12.3575,  15.4669,   3.5240,  -1.0531,  -3.5328]],
       grad_fn=<AddmmBackward>)
predic: [22]


In [72]:
torch.max(outputs.data, 1)[1].cpu().numpy()[0]

22

In [91]:
content = load_dataset("禧贝happybaby 蓝莓甜菜糙米饼40g/袋")
test_iter = build_iterator(test_data, config)
with torch.no_grad():
#     outputs = model(list(test_iter)[0][0])
    outputs = model(content)
    predic = torch.max(outputs.data, 1)[1].cpu().numpy()
    print("outputs:",outputs)
    print("predic:",predic)

outputs: tensor([[ -0.3162,   0.7501,  -6.2322,  -7.3522,  -6.8863,  -6.0743,  -4.0020,
          -3.6948,  -7.3698,   8.7794,  -3.1653,  -4.1497,  -1.3720,   1.6864,
           1.5171,  -7.0695,  -8.6547, -12.9463,  -8.0630,  -6.2699,  -3.6001,
         -12.3575,  15.4669,   3.5240,  -1.0531,  -3.5328]])
predic: [22]


In [84]:
predic

array([19, 24,  8,  4, 25, 20, 13,  0, 21,  1,  5,  4,  7,  1, 24,  2, 21,
        1,  2, 19, 14, 13,  1, 25, 14,  9, 14, 24, 19, 14, 14, 14, 13, 23,
       14,  1, 18, 21, 22, 12, 18, 14, 21, 25,  3,  1,  0, 11, 10, 13, 19,
       10, 19, 22, 20, 14, 19, 25,  1,  0,  3, 22, 13, 24,  1, 23,  0,  0,
       13,  5, 19, 12,  3, 13,  1, 22, 21, 19, 25,  1, 24,  1,  4,  8,  6,
       22, 19, 13,  0, 24,  8, 14, 19, 19,  0, 22, 19, 14,  0, 23,  1, 14,
       14, 25,  3, 19, 13, 24, 12,  7,  0, 20,  1, 19,  8, 12,  7, 22, 11,
        6, 25, 19,  1, 12,  1,  1, 19,  1])

In [92]:
[id_to_cate[i] for i in predic]

['进口食品']

In [82]:
list(test_iter)[0][0]

(tensor([[  11,   45,    6,  ...,  657,   88,   21],
         [  39,   11,   26,  ..., 3453, 3453, 3453],
         [1054,  571, 1149,  ...,  256,   36,    5],
         ...,
         [ 703,  370,  424,  ..., 3453, 3453, 3453],
         [  19,   16,   11,  ..., 3453, 3453, 3453],
         [ 338,  170,  651,  ...,    0,   30,    8]]),
 tensor([32, 19, 32, 32, 18, 25, 32, 21, 18, 32, 18, 32, 21, 24, 32, 24, 27, 23,
         20, 32, 32, 28, 20, 18, 32, 26, 31, 19, 32, 25, 20, 26, 28, 32, 20, 25,
         14, 26, 22, 27, 20, 31, 32, 32, 23, 13, 25, 11, 29, 19, 22, 32, 32, 11,
         30, 21, 32, 32, 23, 24, 32, 16, 32, 24, 10, 15, 11, 23, 32, 24, 32, 19,
         32, 24, 26,  9, 19,  8,  2, 26, 32, 27, 21, 18, 25,  9, 13, 32, 15, 24,
         30, 16, 32, 13, 18, 26, 32, 11, 20, 16, 14, 32, 25, 15, 11, 17, 23, 32,
         19, 26, 24, 21, 22,  4, 32, 26, 17, 20, 32, 25, 30, 16, 17, 32, 24, 25,
         26, 32]))

In [87]:
content

(tensor([  84,   59,    4,   13, 3453, 3453, 3453, 3453, 3453, 3453, 3453, 3453,
         3453, 3453, 3453, 3453, 3453, 3453, 3453, 3453, 3453, 3453, 3453, 3453,
         3453, 3453, 3453, 3453, 3453, 3453, 3453, 3453]), 1)

In [77]:
# coding: UTF-8
import time
import torch
import numpy as np
from train_eval import train, init_network
from importlib import import_module


if __name__ == '__main__':
    dataset = 'dataset'  # 数据集
    embedding = 'random'
    model_name = 'TextCNN'  # 'TextRCNN'  # TextCNN, TextRNN, FastText, TextRCNN, TextRNN_Att, DPCNN, Transformer
    from utils import build_dataset, build_iterator, get_time_dif

    x = import_module(model_name)
    config = x.Config(dataset, embedding)
    np.random.seed(1)
    torch.manual_seed(1)
    torch.cuda.manual_seed_all(1)
    torch.backends.cudnn.deterministic = True  # 保证每次结果一样

    start_time = time.time()
    print("Loading data...")
    vocab, train_data, dev_data, test_data = build_dataset(config, False)
    train_iter = build_iterator(train_data, config)
    dev_iter = build_iterator(dev_data, config)
    test_iter = build_iterator(test_data, config)
    time_dif = get_time_dif(start_time)
    print("Time usage:", time_dif)

3331it [00:00, 33302.73it/s]

Loading data...
Vocab size: 3454


96456it [00:02, 37726.88it/s]
12051it [00:00, 44899.97it/s]
12060it [00:00, 32644.31it/s]

Time usage: 0:00:03


In [52]:
list(test_iter)[0][0]

(tensor([[  11,   45,    6,  ...,  657,   88,   21],
         [  39,   11,   26,  ..., 3453, 3453, 3453],
         [1054,  571, 1149,  ...,  256,   36,    5],
         ...,
         [ 703,  370,  424,  ..., 3453, 3453, 3453],
         [  19,   16,   11,  ..., 3453, 3453, 3453],
         [ 338,  170,  651,  ...,    0,   30,    8]]),
 tensor([32, 19, 32, 32, 18, 25, 32, 21, 18, 32, 18, 32, 21, 24, 32, 24, 27, 23,
         20, 32, 32, 28, 20, 18, 32, 26, 31, 19, 32, 25, 20, 26, 28, 32, 20, 25,
         14, 26, 22, 27, 20, 31, 32, 32, 23, 13, 25, 11, 29, 19, 22, 32, 32, 11,
         30, 21, 32, 32, 23, 24, 32, 16, 32, 24, 10, 15, 11, 23, 32, 24, 32, 19,
         32, 24, 26,  9, 19,  8,  2, 26, 32, 27, 21, 18, 25,  9, 13, 32, 15, 24,
         30, 16, 32, 13, 18, 26, 32, 11, 20, 16, 14, 32, 25, 15, 11, 17, 23, 32,
         19, 26, 24, 21, 22,  4, 32, 26, 17, 20, 32, 25, 30, 16, 17, 32, 24, 25,
         26, 32]))

In [96]:
import pymysql
import random
from sklearn.utils import shuffle
import pandas as pd

conn=pymysql.connect(host='106.75.133.199',port=6823,user='gml_read',passwd='X7CGsW0HBKsdjsG_Msc;af',db='gml_base_qa',use_unicode=True, charset="utf8")
sql="SELECT a.goods_name,a.category_level2,b.name FROM common_goods_info a LEFT JOIN category_info b ON a.category_level2=b.id "
df_all=pd.read_sql(sql,con=conn)
conn.close()
df_model = df_all.loc[df_all["category_level2"].notna(),["goods_name","name"]]
categories = sorted(set(df_model["name"]))
categories_df = pd.DataFrame({"name":categories,"label":range(len(categories))})

In [98]:
df_model.head()

,goods_name,name
0,Schiff旭福预防乳糖益生菌胶囊32粒,保健品
1,拉杜蓝乔La Tourangelle 核桃油 250ml,进口食品
2,禧贝happybaby 蓝莓甜菜糙米饼40g/袋,进口食品
3,Meadjohnson美赞臣 婴幼儿奶粉 安婴宝Enfapro 2段,婴幼儿奶粉
4,Meadjohnson美赞臣 Premium 幼儿配方奶粉 3段,婴幼儿奶粉


In [97]:
categories_df

,name,label
0,个人洗护,0
1,保健品,1
2,口腔护理,2
3,女装/女士内衣,3
4,婴幼儿奶粉,4
5,孕产妇用品,5
6,宝宝服饰/玩具,6
7,宝宝洗护,7
8,宝宝用品_含纸尿片,8
9,宝宝食品,9


In [106]:

import torch
A=torch.ones(2,3) #2x3的张量（矩阵）                                     
B=2*torch.ones(2,4)#4x3的张

In [107]:
print("A:",A)
print("B:",B)

A: tensor([[1., 1., 1.],
        [1., 1., 1.]])
B: tensor([[2., 2., 2., 2.],
        [2., 2., 2., 2.]])


In [108]:
torch.cat((A,B),1)

tensor([[1., 1., 1., 2., 2., 2., 2.],
        [1., 1., 1., 2., 2., 2., 2.]])